# Hackaton | IA para Devs | Fase 5
- Silvio Sales do Nascimento Junior (RM 353303)

## Documentação do Fluxo de Desenvolvimento da Solução

In [56]:
import os
import cv2
import time
import mimetypes
import numpy as np
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from ultralytics import YOLO
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

### Introdução
Esta solução foi desenvolvida para processar vídeos e detectar a presença de objetos cortantes, como facas e tesouras, utilizando o modelo YOLOv8. Ao detectar um objeto, a imagem correspondente é capturada e enviada via e-mail para um destinatário predefinido.

Observação: Utilizei datasets prontos da plataforma Roboflow contendo 700 objetos com facas e treinei um modelo. 

### Passo a Passo

1. **Carregamento do Modelo**
   - O modelo YOLOv8 é carregado para identificar objetos em imagens extraídas de um vídeo.

2. **Criação da Pasta para Armazenamento de Frames**
   - Criamos uma pasta chamada `frames_detectados` para armazenar os frames que contenham objetos suspeitos.

3. **Comparação de Imagens**
   - Implementamos a métrica `Structural Similarity Index (SSIM)` para evitar a captura e envio de imagens muito semelhantes.

4. **Detecção de Objetos**
   - Utilizamos YOLO para verificar se há objetos cortantes nas imagens extraídas.

5. **Envio de Alerta por E-mail**
   - Caso um objeto cortante seja detectado, um e-mail é enviado com a imagem anexada.

6. **Processamento de Vídeo**
   - O vídeo é lido frame a frame, aplicando a lógica de detecção e envio de alerta sempre que necessário.

### Código Fonte

In [57]:
# Carregar modelo treinado
yolo_model = YOLO("modelo_treinado.pt")

# Criar pasta para armazenar frames detectados
output_folder = "frames_detectados"
os.makedirs(output_folder, exist_ok=True)

# Variáveis globais para armazenar o último frame detectado e sua imagem
last_detected_image = None  # Matriz NumPy do último frame salvo

detection_log = []  # Lista para armazenar registros de detecção

#### Função para calcular a similaridade entre dois frames

In [58]:
def is_similar(image1, image2, threshold=0.90):
    """Compara duas imagens e retorna True se forem muito semelhantes."""
    if image1 is None or image2 is None:
        return False  # Primeira imagem, sempre aceitar

    # Convertendo imagens para tons de cinza
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Redimensionando para garantir tamanhos iguais
    gray1 = cv2.resize(gray1, (gray2.shape[1], gray2.shape[0]))

    # Calculando a similaridade entre as imagens
    score, _ = ssim(gray1, gray2, full=True)
    return score >= threshold  # Retorna True se forem muito semelhantes

### Função para processar imagem e detectar objetos

In [59]:
def detectar_objetos(image_path):
    image = Image.open(image_path)
    results = yolo_model(image)

    for result in results:
        for box in result.boxes:
            cls = result.names[int(box.cls[0])]
            if cls in ["knife", "scissors"]:  # Ajuste para as classes do dataset
                return True  # Objeto cortante detectado
    return False

#### Função para enviar alerta por e-mail com anexo

In [60]:
def enviar_alerta(image_path, tempo):
    sender = "silviosnjr@yahoo.com.br"
    receiver = "silviosnjr@gmail.com"
    subject = "Alerta de Segurança: Objeto Cortante Detectado"
    body = f"Objeto cortante detectado no tempo {tempo:.2f} segundos Veja a imagem em anexo."

    # Criando e-mail com suporte para anexos
    msg = MIMEMultipart()
    msg["Subject"] = subject
    msg["From"] = sender
    msg["To"] = receiver
    msg.attach(MIMEText(body, "plain"))

    # Anexando a imagem
    try:
        with open(image_path, "rb") as img_file:
            img_data = img_file.read()
            mime_type, _ = mimetypes.guess_type(image_path)
            if mime_type is None:
                mime_type = "application/octet-stream"

            main_type, sub_type = mime_type.split("/", 1)
            img_attachment = MIMEImage(img_data, _subtype=sub_type)
            img_attachment.add_header("Content-Disposition", f"attachment; filename={os.path.basename(image_path)}")
            msg.attach(img_attachment)
    except Exception as e:
        print(f"Erro ao anexar a imagem: {e}")
        return

    # Configuração do servidor SMTP do Yahoo
    smtp_server = "smtp.mail.yahoo.com"
    smtp_port = 587  # Usando TLS
    email_password = os.getenv("SENHA_YAHOO")  # Obtendo senha do ambiente

    try:
        with SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Ativa criptografia TLS
            server.login(sender, email_password)  # Login com e-mail e senha de app
            server.sendmail(sender, receiver, msg.as_string())
        print("Alerta enviado com sucesso!")
    except Exception as e:
        print("Erro ao enviar e-mail:", e)

#### Processamento de vídeo gravado

In [ ]:
video_path = "videos/video2.mp4"  # Substituir pelo caminho do vídeo
cap = cv2.VideoCapture(video_path)

frame_rate = cap.get(cv2.CAP_PROP_FPS)  # Obtém a taxa de frames por segundo
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Armazendo frame
    frame_count += 1
    timestamp = frame_count / frame_rate  # Calcula o tempo em segundos no vídeo
    image_path = os.path.join(output_folder, f"frame_{int(time.time())}.jpg")

    # Evitar salvar e-mails para frames semelhantes
    if last_detected_image is None or not is_similar(last_detected_image, frame, threshold=0.90):
        cv2.imwrite(image_path, frame)

        if detectar_objetos(image_path):
            detection_log.append(f"Objeto detectado no tempo: {timestamp:.2f} segundos")
            last_detected_image = frame  # Atualiza a última imagem detectada
            print(f"Objeto cortante detectado! Frame salvo em {image_path} - Tempo: {timestamp:.2f}s")
            enviar_alerta(image_path, timestamp)
        else:
            os.remove(image_path)  # Remove o frame salvo se não houver detecção
    else:
        print("Frame muito semelhante ao anterior, ignorado.")

    cv2.imshow("Video", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#### Criar relatório de detecção

In [ ]:
report_path = "relatorio/relatorio_deteccao.txt"
with open(report_path, "w") as report_file:
    report_file.write("Relatorio de Deteccao de Objetos Cortantes\n")
    report_file.write("=" * 50 + "\n")
    report_file.write("\n".join(detection_log))

print(f"Relatorio salvo em {report_path}")

### Conclusão
Este notebook apresenta uma solução eficiente para detecção de objetos cortantes em vídeos. O modelo YOLOv8 é utilizado para identificar facas e tesouras, enquanto um mecanismo de comparação evita redundâncias. O envio de alertas por e-mail garante uma resposta rápida a eventos suspeitos.